In [1]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn

import os 
import fastai
import fastai
from fastai.text import *
PATH="/home/kirana/Documents/phd/final"
DATAPATH="/home/kirana/Documents/phd/data/aclImdb"

## Dataset

In [2]:
[df_train,df_valid,df_test,itos, train_tokens, valid_tokens, test_tokens, trn_lm, val_lm, test_lm]=pickle.load(open(f'{PATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [3]:
bs=52 # 52 - Jeremey, 20 - default

In [4]:
bptt= 70 #70 - Jeremey, 35 - default

In [5]:
df_train.head()

,label,text,words,tokens
18413,2,I've just got back from watching this delightf...,"[ \n , xxbos, xxfld, 1, i, 've, just, got, bac...","[41, 42, 43, 40, 13, 161, 57, 210, 162, 51, 16..."
43883,2,"For a first film, this shows a great deal of v...","[ \n , xxbos, xxfld, 1, xxmaj, for, a, first, ...","[41, 42, 43, 40, 2, 24, 7, 106, 27, 5, 14, 298..."
45342,2,I am fully aware that no film adaptation of a ...,"[ \n , xxbos, xxfld, 1, i, am, fully, aware, t...","[41, 42, 43, 40, 13, 264, 1390, 1811, 15, 74, ..."
6454,2,I'm always very cautious when people refer to ...,"[ \n , xxbos, xxfld, 1, i, 'm, always, very, c...","[41, 42, 43, 40, 13, 167, 236, 70, 15406, 69, ..."
13703,2,I started to watch this movie but gave up beca...,"[ \n , xxbos, xxfld, 1, i, started, to, watch,...","[41, 42, 43, 40, 13, 677, 9, 127, 14, 25, 26, ..."


In [6]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', 'the', '.']

In [7]:
n_batch=df_train.shape[0]//bs
padlen=640
padding_idx=1

In [8]:
df_train['n_tok']=df_train['tokens'].apply(len)

In [9]:
df_train['n_tok'].describe()

count    67500.000000
mean       305.567867
std        225.163517
min         15.000000
25%        166.000000
50%        229.000000
75%        371.000000
max       3455.000000
Name: n_tok, dtype: float64

In [10]:
df_train.sort_values(by='n_tok', inplace=True)

In [11]:
df_valid['n_tok']=df_valid['tokens'].apply(len)
df_valid.sort_values(by='n_tok', inplace=True)

In [12]:
df_test['n_tok']=df_test['tokens'].apply(len)
df_test.sort_values(by='n_tok', inplace=True)

In [13]:
for i in range(0,n_batch):
    x=df_train['tokens'].values[i]
    y=df_train['label'].values[i]    
    out=np.ones(padlen)
    if len(x) < padlen:
        out[:len(x)]=x
    else:
        out=x[:padlen]
    print (out)
    print (y)
    print (len(out))
    if i>10:
        break

[41. 42. 43. 40. ...  1.  1.  1.  1.]
2
640
[41. 42. 43. 40. ...  1.  1.  1.  1.]
2
640
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
640
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
640
[41. 42. 43. 40. ...  1.  1.  1.  1.]
2
640
[41. 42. 43. 40. ...  1.  1.  1.  1.]
2
640
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
640
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
640
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
640
[41. 42. 43. 40. ...  1.  1.  1.  1.]
2
640
[41. 42. 43. 40. ...  1.  1.  1.  1.]
2
640
[41. 42. 43. 40. ...  1.  1.  1.  1.]
2
640


In [14]:
df_train.shape

(67500, 5)

In [15]:
df_train['label'].value_counts(), df_valid['label'].value_counts(),df_test['label'].value_counts()

(2    45002
 1    11252
 0    11246
 Name: label, dtype: int64, 2    4998
 0    1254
 1    1248
 Name: label, dtype: int64, 1    12500
 0    12500
 Name: label, dtype: int64)

In [16]:
df_train_big=df_train.copy()

In [17]:
df_valid_big=df_valid.copy()

In [18]:
df_valid=df_valid.loc[df_valid['label']<2]

In [19]:
df_train=df_train.loc[df_train['label']<2]

In [20]:
df_train.shape, df_valid.shape

((22498, 5), (2502, 5))

In [21]:
np.ones(3).astype(int)

array([1, 1, 1])

In [22]:
class ds_sentiment (Dataset):
    def __init__ (self,df,bs,padlen=64,xvar='tokens',yvar='label',len_var='n_tok',padding_idx=1):
        self.x,self.y,self.padlen,self.padding_idx,self.len_var,self.bs=\
            df[xvar],df[yvar],padlen,padding_idx,df[len_var],bs
    def pad (self,x):
        out=np.ones(self.padlen)*self.padding_idx
        out=out.astype(int)
        if len(x)>=self.padlen:
            out[:]=x[:self.padlen]
        else:
            out[:len(x)]=x
        return out
        
    def __getitem__(self,idx):
        #import pdb
        #pdb.set_trace()
        return self.pad(self.x.iloc[idx]),self.y.iloc[idx],self.len_var.iloc[idx]
    
    def __len__(self):
        return len(self.x)        


In [23]:
padlen=max(df_train['n_tok'])

In [24]:
padlen

3354

In [25]:
dstrain=ds_sentiment(df_train,bs,padlen)
dsvalid=ds_sentiment(df_valid,bs,padlen)
dstest=ds_sentiment(df_test,bs,padlen)

In [26]:
dltrain=DataLoader(dstrain,bs,True)
dlvalid=DataLoader(dsvalid,bs,False)
dltest=DataLoader(dstest,bs,False)

In [27]:
for xb,yb,xlen in dltrain:
    break

In [28]:
xb, yb, xlen

(tensor([[41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         ...,
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1]]),
 tensor([0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1,
         0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
         1, 1, 0, 0]),
 tensor([ 171,  250,  223,  165,  143,  389,  447,  192,  328,  124,  304,  163,
          259,   80,  164,  135,  283,  191, 1017,  124,  426,  146,  314,  199,
          186,  216,  245,  154,  664,  360,  218,   85,  107,  126,  185,  267,
           61,  436,  113,  259,  479,  195,  330,  183,   73,  513,   95,  200,
          162,  188,   98,  178]))

## Model Architecture

In [29]:
from torch import nn
import torch
from torch.autograd import Variable

In [30]:
bs,bptt

(52, 70)

In [35]:
n_inp=len(itos)
n_emb=400 #650
n_hidden=400 #650
n_layers= 2 # 2
dropout= 0.5
wd=1e-7
bidirectional=False
dropout_e= 0.05
dropout_o=0.5
n_out=1
n_hidden2=25

In [36]:
xb.shape,xlen.shape

(torch.Size([52, 3354]), torch.Size([52]))

In [37]:
df_train['label'].value_counts()

1    11252
0    11246
Name: label, dtype: int64

In [38]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [39]:
class sentiment_classifier (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,n_out=1,padding_idx=1,n_hidden2=25):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx,self.padding_idx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx,padding_idx
        self.n_out,self.n_hidden2=n_out,n_hidden2
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        
        self.create_architecture()
        if pretrain_mtx is not None:
            print ("initializing")
            self.initialize_glove()
        self.init_hidden()
        self.criterion=nn.BCELoss()
        
    def create_architecture(self):
        # Dropout layer
        self.dropout_enc=nn.Dropout(self.dropout_e)
        # Embedding Layer: Embedding layer just maps each word to an index. n_inp to n_emb mapping is all it does
            # input to this is of shape n_batch * n_seq
        self.encoder=nn.Embedding(self.n_inp,self.n_emb,padding_idx=self.padding_idx)
             # embs are going to be of shape n_batch * n_seq * n_emb
            # h and c are of shape n_layers * n_batch * n_hidden
        # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=self.bidirectional)
        
                # output is of shape n_batch * n_seq * n_hidden
            # output [:,-1,:] is the same as hn[-1]
        self.dropout_op=nn.Dropout(self.dropout_o)
        self.fc=nn.Linear(self.n_hidden,self.n_out)

        #self.log_softmax=nn.LogSoftmax()
        self.log_softmax=nn.Sigmoid()
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
         
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True

    def initialize_glove(self):
        self.encoder.weight.data=torch.Tensor(self.pretrain_mtx)
    
    def init_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.randn(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.randn(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    
        
    def forward(self,Xb,Yb,Xb_lengths):
        embs=self.encoder(Xb)
        #embs=self.dropout_enc(embs)
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())
        
        # before lstm call
            # pack_padded_sequence so that padded items in the sequence won't be shown to the LSTM
        embs = pack_padded_sequence(embs,Xb_lengths,batch_first=True,enforce_sorted=False)
        lstm_out,self.hidden=self.lstm(embs,self.hidden)

        # after lstm call
        lstm_out,_=pad_packed_sequence(lstm_out,batch_first=True)
        #lstm_out=self.dropout_op(lstm_out)

        #import pdb
        #pdb.set_trace()
       
        
        # Wrap the hidden state in a new tensor without the gradients
        if 1==1:
            self.hidden=(Variable(self.hidden[0].data,requires_grad=False).to(self.device),\
                         Variable(self.hidden[1].data,requires_grad=False).to(self.device))
        
            # lstm_out has the values that are padded
        #out=self.fc (lstm_out[:,-1,:])
        out=self.fc(self.hidden[0][-1])

        preds=self.log_softmax(out)
        
              
        
        loss=self.criterion(preds,Yb.contiguous().float().view(-1,1))
        
        return preds.view(-1), loss  

In [40]:
def accuracy_binomial(preds,actual):
    correct=torch.round(preds).long()==actual
    return correct.float().sum()/len(correct)

In [41]:
device="cpu"

## Load pretrained model weights

In [42]:
pretrained_lm_weights=pickle.load(open(f'{PATH}/inter/pretrained_lm_weights','rb'))

In [43]:
if 1==0:
    model=torch.load (f'{PATH}/inter/model_awd_lstm')
    pretrained_lm_weights=model.encoder.weight.data.cpu().numpy()
    import pickle
    pickle.dump(pretrained_lm_weights,open(f'{PATH}/inter/pretrained_lm_weights','wb'))

### Check if model forward works

In [44]:
n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout_o,dropout,n_out

(59972, 400, 400, 2, False, 52, 'cpu', 0.05, 0.5, 0.5, 1)

In [45]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=dropout_e,dropout=dropout,\
                 dropout_o=dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1,n_hidden2=100)


initializing


In [46]:
model_sentiment

sentiment_classifier(
  (dropout_enc): Dropout(p=0.05)
  (encoder): Embedding(59972, 400, padding_idx=1)
  (lstm): LSTM(400, 400, num_layers=2, batch_first=True, dropout=0.5)
  (dropout_op): Dropout(p=0.5)
  (fc): Linear(in_features=400, out_features=1, bias=True)
  (log_softmax): Sigmoid()
  (criterion): BCELoss()
)

In [47]:
model_sentiment.forward(xb,yb,xlen)

(tensor([0.5010, 0.5039, 0.5030, 0.5010, 0.5001, 0.4957, 0.5024, 0.4984, 0.4988,
         0.4995, 0.4981, 0.5003, 0.5002, 0.4997, 0.4973, 0.4975, 0.4982, 0.4953,
         0.4961, 0.4971, 0.5035, 0.5019, 0.5017, 0.4968, 0.4970, 0.4990, 0.5038,
         0.4983, 0.5024, 0.5023, 0.4942, 0.4989, 0.4976, 0.5033, 0.4997, 0.4993,
         0.4937, 0.5017, 0.4960, 0.4981, 0.5002, 0.4992, 0.5021, 0.5003, 0.4985,
         0.5019, 0.4975, 0.4958, 0.5077, 0.5006, 0.5039, 0.5019],
        grad_fn=<ViewBackward>),
 tensor(0.6949, grad_fn=<BinaryCrossEntropyBackward>))

In [48]:
preds,loss=model_sentiment.forward(xb,yb,xlen)

In [49]:
preds.view(-1)

tensor([0.4982, 0.4975, 0.5049, 0.4989, 0.5002, 0.4945, 0.5047, 0.4988, 0.5027,
        0.5007, 0.5002, 0.5034, 0.4985, 0.4996, 0.5016, 0.5038, 0.5022, 0.5005,
        0.5000, 0.5028, 0.4989, 0.5015, 0.4965, 0.4985, 0.4984, 0.5038, 0.5001,
        0.5038, 0.4998, 0.5041, 0.4971, 0.4956, 0.4950, 0.5028, 0.4977, 0.4973,
        0.4975, 0.5020, 0.4954, 0.4986, 0.4978, 0.5010, 0.4990, 0.4997, 0.4982,
        0.4987, 0.4996, 0.4961, 0.5006, 0.4981, 0.5073, 0.5007],
       grad_fn=<ViewBackward>)

In [50]:
yb

tensor([0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
        1, 1, 0, 0])

In [51]:
preds.size()

torch.Size([52])

In [52]:
yb.size()

torch.Size([52])

In [53]:
accuracy_binomial(preds,yb)

tensor(0.5000)

In [54]:
from sklearn.metrics import roc_auc_score

In [55]:
roc_auc_score(yb,preds)

0.3851851851851852

In [56]:
pretrained_lm_weights

array([[-9.038397e-02, -1.795423e-02, -3.457930e-02,  8.671192e-02, ..., -3.003510e-02, -1.899482e-02,  1.426260e-02,
        -4.948893e-02],
       [-2.453323e-02, -3.018817e-02,  6.663179e-02, -5.793565e-02, ...,  1.840935e-02,  6.371137e-02,  9.835491e-03,
        -2.131385e-03],
       [ 7.696263e-03, -4.266643e-02, -1.533517e-01,  2.239776e-01, ...,  9.869517e-02,  3.041433e-02,  1.824751e-01,
         1.134978e-01],
       [ 3.305928e-02,  2.266591e-01, -4.264669e-02,  1.490862e-01, ...,  3.407921e-02, -6.422209e-03,  3.180612e-01,
         9.373549e-02],
       ...,
       [ 6.027538e-02, -5.980809e-02,  1.861691e-01, -3.105092e-02, ..., -2.764457e-02,  1.962678e-02, -2.172215e-03,
         6.297247e-02],
       [-1.592789e-02, -2.781571e-04,  1.301994e-01,  2.851282e-02, ...,  5.064877e-02,  1.670864e-01,  2.283701e-02,
        -8.746398e-03],
       [ 4.669700e-02,  3.138980e-02,  1.221957e-02, -3.927753e-02, ..., -1.281436e-01,  1.121320e-01, -3.726090e-03,
        -3.777364e

## Learner

In [57]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1
  
        
    
    def fit (self,Xb,Yb,Xlen,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb,Xlen)
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1))
            acc=acc.item()
            del preds
        
        if mode_train:
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.clip_val)
            if 1==0:
                for p in self.model.parameters():
                    p.data.add_(self.lr, p.grad.data)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        epoch_loss,epoch_acc,i,k=0,0,0,0
        self.model.init_hidden()
        for Xb,Yb,Xlen in iterator:
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            Xlen=Xlen.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,Xlen,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            
            k=k+1
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
        epoch_loss=epoch_loss/k
        epoch_acc=epoch_acc/k
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

    
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        for epoch in range(n_epochs):
            
            loss,acc=self.run_epoch(dltrain,True)
            print (f'Epoch:{epoch} Loss:{loss}')
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Loss:{loss} Accuracy:{acc} Loss:{lossv} Accuracy:{accv}')
        

In [58]:
device="cuda:0"

In [59]:
n_layers

2

In [60]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout,\
                 dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1,n_hidden2=n_hidden2)
model_sentiment=model_sentiment.to(device)

initializing


In [61]:
optimizer=torch.optim.Adam(model_sentiment.parameters(),lr=1e-3,betas=(0.9,0.999), weight_decay=wd)
metric_fn=accuracy_binomial

In [62]:
len(dltrain),len(dlvalid)

(433, 49)

In [63]:
df_valid['label'].value_counts()

0    1254
1    1248
Name: label, dtype: int64

In [64]:
learner=Learner(model_sentiment,optimizer,accuracy_binomial,device,bptt,50,0.25)

In [65]:
model_sentiment.freeze_embedding()

In [66]:
model_sentiment.encoder.weight.requires_grad

False

In [67]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.6929491329193115  0.5080769407749176
Batch:100 0.6929148989915848  0.5069230940937995
Batch:150 0.6926568806171417  0.5157692476113638
Batch:200 0.6923693001270295  0.5208654017746448
Batch:250 0.6921350879669189  0.522692324757576
Batch:300 0.6919598573446274  0.5255769407749176
Batch:350 0.6917337763309479  0.5302197984286717
Batch:400 0.6915489307045937  0.5310096336156129
Epoch:0 Loss:0.6914546627370645
Epoch:0 Loss:0.6914546627370645 Accuracy:0.5321549301059362 Loss:0.6900792377335685 Accuracy:0.5256410447918639


In [68]:
for param_group in learner.optimizer.param_groups:
    param_group['lr']=5e-3

In [69]:
len(dltrain)

433

In [70]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.6888651859760284  0.5465384829044342
Batch:100 0.6868347001075744  0.5592307892441749
Batch:150 0.6865313390890757  0.5566666867335638
Batch:200 0.6862078475952148  0.5586538667976856
Batch:250 0.6858403780460358  0.5570769445896149
Batch:300 0.6851726808150609  0.5599359187483788
Batch:350 0.6844567174570901  0.5637912304060799
Batch:400 0.6838803415000438  0.5630769443511963
Epoch:0 Loss:0.6838188163127414
Epoch:0 Loss:0.6838188163127414 Accuracy:0.5624680177719433 Loss:0.6815139103908928 Accuracy:0.5648875157443844


In [71]:
learner.run_epochs(dltrain,dlvalid,3)

Batch:50 0.6763112819194794  0.5957692557573319
Batch:100 0.677610512971878  0.5861538690328598
Batch:150 0.6767759549617768  0.5900000228484472
Batch:200 0.6763600370287896  0.5888461762666702
Batch:250 0.677184455871582  0.5840000220537186
Batch:300 0.6770491216580073  0.5844872006773949
Batch:350 0.6766948577335903  0.5839011204242707
Batch:400 0.6763908997178077  0.5843269442021847
Epoch:0 Loss:0.6763449871512539
Epoch:0 Loss:0.6763449871512539 Accuracy:0.5839168172647992 Loss:0.6754414633828767 Accuracy:0.594584006436017
Batch:50 0.6701778411865235  0.590384635925293
Batch:100 0.6711819350719452  0.5898077124357224
Batch:150 0.6716099389394125  0.5919230979681015
Batch:200 0.6717160579562187  0.5920192520320415
Batch:250 0.6719906995296479  0.589692329287529
Batch:300 0.6724682903289795  0.585705149670442
Batch:350 0.671772335256849  0.5873626582111631
Batch:400 0.672002954185009  0.5861538670212031
Epoch:1 Loss:0.6721237377536765
Epoch:1 Loss:0.6721237377536765 Accuracy:0.5862994

In [182]:
for param_group in learner.optimizer.param_groups:
    param_group['lr']=1e-4

In [183]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.64853853225708  0.6207692527770996
Batch:100 0.6483927577733993  0.6265384835004807
Batch:150 0.6484267862637838  0.6321795090039571
Batch:200 0.6477748197317124  0.6351923295855522
Batch:250 0.6486767339706421  0.6325384842157364
Batch:300 0.6489897185564041  0.6337179721395174
Batch:350 0.6487747352463858  0.6352198032821927
Batch:400 0.6485072585940361  0.6359134846925736
Epoch:0 Loss:0.648502684768298
Epoch:0 Loss:0.648502684768298 Accuracy:0.6363788593999232 Loss:0.655621100445183 Accuracy:0.626373649859915


In [184]:
for param_group in learner.optimizer.param_groups:
    param_group['lr']=1e-3

In [185]:
model_sentiment.unfreeze_embedding()

In [186]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.6453262722492218  0.6419231033325196
Batch:100 0.646092454791069  0.6440384846925735
Batch:150 0.6475411713123321  0.6400000242392222
Batch:200 0.6472525778412819  0.6382692554593086
Batch:250 0.6482151463031769  0.6372307926416397
Batch:300 0.648103004693985  0.6371154076854388
Batch:350 0.6477661645412445  0.637967055950846
Batch:400 0.647689622193575  0.6379807917028666
Epoch:0 Loss:0.6474101744402914
Epoch:0 Loss:0.6474101744402914 Accuracy:0.6383487004743713 Loss:0.6549209563099608 Accuracy:0.6357928058322595


In [187]:
learner.run_epochs(dltrain,dlvalid,3)

Batch:50 0.6463013350963592  0.6476923274993897
Batch:100 0.6445563387870789  0.6455769440531731
Batch:150 0.6444319272041321  0.6460256640116374
Batch:200 0.6456667852401733  0.642500022649765
Batch:250 0.6460841224193573  0.6393846383094788
Batch:300 0.6460355154673258  0.6408333570758502
Batch:350 0.6468912330695561  0.6384066168751036
Batch:400 0.6467960438132286  0.6388461776077747
Epoch:0 Loss:0.6463942212540889
Epoch:0 Loss:0.6463942212540889 Accuracy:0.6399475645798733 Loss:0.6536961599272124 Accuracy:0.6306907619748797
Batch:50 0.6493575870990753  0.6342307943105697
Batch:100 0.647639335989952  0.6423077157139778
Batch:150 0.6473417774836222  0.6419231015443801
Batch:200 0.6454289036989213  0.6459615634381771
Batch:250 0.6455069897174835  0.6428461775779725
Batch:300 0.6450929262240728  0.6438461778561274
Batch:350 0.6450396980558123  0.6428571672098977
Batch:400 0.6445977027714253  0.6433173318207264
Epoch:1 Loss:0.6451607778771645
Epoch:1 Loss:0.6451607778771645 Accuracy:0.6

In [86]:
learner.device=device
learner.model=learner.model.to(device)
model.device=device
learner.model.device=device
learner.model.hidden=(learner.model.hidden[0].cuda(1),learner.model.hidden[1].cuda(1))
model.hidden=(model.hidden[0].cuda(1),model.hidden[1].cuda(1))


In [92]:
def set_gpu_recursive(var, gpu_id):
    for key in var:
        if isinstance(var[key], dict):
            var[key] = set_gpu_recursive(var[key], gpu_id)
        else:
            try:
                var[key] = var[key].cuda(gpu_id)
            except:
                pass
    return var



In [100]:
optimizer.state = set_gpu_recursive(optimizer.state, 1)
learner.optimizer.state = set_gpu_recursive(learner.optimizer.state, 1)

In [103]:
learner.run_epochs(train_tokens,valid_tokens,1)

Batch:500 4.215455349445343  0.2790599019825459
Batch:1000 4.210862180948258  0.2788794076293707
Batch:1500 4.212145571072896  0.2783448839088281
Batch:2000 4.211311246871948  0.278226111009717
Batch:2500 4.210199825286865  0.27802792423963546
Batch:3000 4.209871447563171  0.27792235642671587
Batch:3500 4.2090058584213255  0.27796406232459203
Batch:4000 4.207844992935658  0.2780102457590401
Batch:4500 4.207080701986949  0.27803181937999194
Batch:5000 4.206981196069718  0.2780740782529116
Batch:5500 4.205610530679876  0.27823452775315805
Epoch:0 Loss:4.20557110817913
Batch:500 3.911064175128937  0.30575276267528534
Epoch:0 Loss:4.20557110817913 Accuracy:0.27822952477167406 Loss:3.910363855164207 Accuracy:0.30540491747513915


In [104]:
np.exp(3.9), np.exp(3.91)

(49.40244910553017, 49.89895197340787)

In [106]:
torch.save(model.state_dict(),f'{PATH}/inter/model_awd_lstm_state_dict_cuda_1')
torch.save(optimizer.state_dict(),f'{PATH}/inter/learner_awd_lstm_state_dict_cuda_1')


In [107]:
torch.save (model,f'{PATH}/inter/model_awd_lstm_cuda_1')


In [108]:
torch.save (optimizer,f'{PATH}/inter/optimizer_awd_lstm_cuda_1')


In [109]:
torch.save (learner,f'{PATH}/inter/learner_awd_lstm_cuda_1')

## Transfer Learning to predict RNN